In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

'/home/vco/Projects/delay-deadline-miner'

In [3]:
import pandas as pd
from datetime import timedelta
import pm4py
from pm4py.algo.discovery.dcr_discover.extenstions.nesting import NestVariants
from collections import defaultdict
from copy import deepcopy

In [26]:
from notebook.ebda_util import *

In [13]:
from pm4py.objects.conversion.log import converter as log_converter
import pandas as pd

cids = [1, 1, 1, 1, 1,
        2, 2, 2, 2, 2,
        3, 3, 3, 3,
        4, 4, 4, 4, 4]
events = ['Doctor requests home care', 'Send mail', 'Consent', 'Assessment', 'Approved',
          'Doctor requests home care', 'Send mail', 'Consent', 'Assessment', 'Denied',
          'Citizen requests home care', 'Consent', 'Assessment', 'Approved',
          'Citizen requests home care', 'Consent', 'Assessment', 'Assessment', 'Denied']
timestamps = [pd.to_datetime('2025-01-01'), pd.to_datetime('2025-01-02'), pd.to_datetime('2025-01-09'),
              pd.to_datetime('2025-01-10'), pd.to_datetime('2025-01-17'),
              pd.to_datetime('2025-01-01'), pd.to_datetime('2025-01-07'), pd.to_datetime('2025-01-18'),
              pd.to_datetime('2025-04-01'), pd.to_datetime('2025-04-05'),
              pd.to_datetime('2025-01-01'), pd.to_datetime('2025-01-07'), pd.to_datetime('2025-01-15'),
              pd.to_datetime('2025-01-21'),
              pd.to_datetime('2025-01-01'), pd.to_datetime('2025-01-05'), pd.to_datetime('2025-01-20'),
              pd.to_datetime('2025-01-27'), pd.to_datetime('2025-02-01')]

test_log_dict = {
    'case:concept:name': cids,
    'concept:name': events,
    'time:timestamp': timestamps
}
test_log_df = pd.DataFrame(test_log_dict)
test_event_log = log_converter.apply(test_log_df)

cids = [1, 1, 1, 1, 1,
        2, 2, 2, 2, 2,
        3, 3, 3, 3,
        4, 4, 4, 4, 4]
events = ['Request', 'Send mail', 'Consent', 'Assessment', 'Decision',
          'Request', 'Send mail', 'Consent', 'Assessment', 'Decision',
          'Request', 'Consent', 'Assessment', 'Decision',
          'Request', 'Consent', 'Assessment', 'Assessment', 'Decision']
timestamps = [pd.to_datetime('2025-01-01'), pd.to_datetime('2025-01-02'), pd.to_datetime('2025-01-09'),
              pd.to_datetime('2025-01-10'), pd.to_datetime('2025-01-17'),
              pd.to_datetime('2025-01-01'), pd.to_datetime('2025-01-07'), pd.to_datetime('2025-01-18'),
              pd.to_datetime('2025-04-01'), pd.to_datetime('2025-04-05'),
              pd.to_datetime('2025-01-01'), pd.to_datetime('2025-01-07'), pd.to_datetime('2025-01-15'),
              pd.to_datetime('2025-01-21'),
              pd.to_datetime('2025-01-01'), pd.to_datetime('2025-01-05'), pd.to_datetime('2025-01-20'),
              pd.to_datetime('2025-01-27'), pd.to_datetime('2025-02-01')]

test_log_dict = {
    'case:concept:name': cids,
    'concept:name': events,
    'time:timestamp': timestamps
}
test_abstracted_log_df = pd.DataFrame(test_log_dict)
test_abstracted_log = log_converter.apply(test_abstracted_log_df)

In [14]:
abstraction_dict = {
    'Request': {'Doctor requests home care', 'Citizen requests home care'},
    'Decision': {'Approved', 'Denied'}
}

abstracted_log_df = abstract_events(test_log_df, abstraction_dict)

In [15]:
abstracted_log_df

,case:concept:name,concept:name,time:timestamp
0,1,Request,2025-01-01
1,1,Send mail,2025-01-02
2,1,Consent,2025-01-09
3,1,Assessment,2025-01-10
4,1,Decision,2025-01-17
5,2,Request,2025-01-01
6,2,Send mail,2025-01-07
7,2,Consent,2025-01-18
8,2,Assessment,2025-04-01
9,2,Decision,2025-04-05


In [27]:
timed_original_dcr, _ = pm4py.discover_dcr(test_event_log, post_process={'timed'})
# nesting_original_dcr, _ = pm4py.discover_dcr(test_event_log, post_process={'nesting'},nest_variant=NestVariants.CHOICE)
original_dcr, _ = pm4py.discover_dcr(test_event_log, post_process={'nesting','timed'},nest_variant=NestVariants.CHOICE)
abstracted_dcr, _ = pm4py.discover_dcr(test_abstracted_log, post_process={'nesting','timed'},nest_variant=NestVariants.CHOICE)

In [28]:
rename_map = {"Choice1": "Decision", "Choice2": "Request"}
renamed_dcr = rename_events_in_dcr_graph(original_dcr, rename_map)

In [29]:
merged_dcr = merge_dcr_graphs(renamed_dcr, abstracted_dcr)

In [30]:
timed_original_dcr = handle_timed_condition_subprocess(timed_original_dcr)
original_dcr = handle_timed_condition_subprocess(original_dcr)
abstracted_dcr = handle_timed_condition_subprocess(abstracted_dcr)
merged_dcr = handle_timed_condition_subprocess(merged_dcr)

In [31]:
from pm4py.objects.dcr.exporter import exporter as dcr_exporter

parameters = {"time_precision":"D"}

dcr_exporter.apply(timed_original_dcr,
                   '/home/vco/Projects/delay-deadline-miner/models/timed_original_dcr.xml',**parameters)
dcr_exporter.apply(original_dcr,
                   '/home/vco/Projects/delay-deadline-miner/models/original_dcr.xml',**parameters)
dcr_exporter.apply(abstracted_dcr,
                   '/home/vco/Projects/delay-deadline-miner/models/abstracted_dcr.xml',**parameters)
dcr_exporter.apply(merged_dcr,
                   '/home/vco/Projects/delay-deadline-miner/models/merged_dcr.xml',**parameters)

/home/vco/Projects/delay-deadline-miner/pm4py/objects/dcr/exporter/variants/xml_simple.py:29: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  if time.floor(freq='S').to_numpy() > 0:
/home/vco/Projects/delay-deadline-miner/pm4py/objects/dcr/exporter/variants/xml_simple.py:31: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  iso_time = time.floor(freq='S').isoformat()
/home/vco/Projects/delay-deadline-miner/pm4py/objects/dcr/exporter/variants/xml_simple.py:45: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  if time.floor(freq='S').to_numpy() > 0:
/home/vco/Projects/delay-deadline-miner/pm4py/objects/dcr/exporter/variants/xml_simple.py:47: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  iso_time = time.floor(freq='S').isoformat()


# Overall idea and selling point
Mine time as the minimum delay and maximum deadline.
Mine the lifecycle of activities.
Mine choices.
Create time aware subprocesses by finding patterns:
- Self exclude, condition: required to respect timed execution semantics
- Conditions and responses between steps in the transition lifecycle for activites: The activity subprocess
Abstract away based on subprocesses:
- Remove individual choices and mine the step of making a choice: downside, the individual delays/deadlines or consequences of a choice are removed. Only the effect of making a choice is kept.
- Remove the lifecycle transition that is not equal to complete: downside, the individual effects of lifecycle transitions are not visible.
Merge the abstract log and original log: best of both worlds. see the effect of the subprocess but also the effect of the individual event.

In [ ]:
log_metadata = pd.read_csv('/home/vco/Datasets/Collection_Event_Logs/Metadata.csv')
log_metadata.columns = log_metadata.columns.str.replace(r"[\s\t\n]+", "", regex=True)

In [ ]:
log_metadata

In [ ]:
suitable_logs = log_metadata[log_metadata['AdditionalAttributes'].str.contains('Lifecycle Transition', na=False)]

In [ ]:
import numpy as np

# Clean base name
base_name = (
    suitable_logs['EventLogName']
    .str.replace(' ', '_', regex=False)
    .str.replace(',', '', regex=False)
)

# Append dataset name only if not NaN
dataset_part = suitable_logs['EventLogDatasetFileName'].fillna('')
dataset_part = np.where(dataset_part != '', '_' + dataset_part, '')

# Conditionally add DatasetFormat only if not already in EventLogDatasetFileName
add_format = ~suitable_logs.apply(
    lambda row: str(row['DatasetFormat']) in str(row['EventLogDatasetFileName']),
    axis=1
)
format_part = np.where(add_format, suitable_logs['DatasetFormat'], '')

# Final column assignment
suitable_logs['EventLogFileName'] = base_name + dataset_part + format_part

suitable_logs_list = suitable_logs['EventLogFileName'].tolist()

In [ ]:
import glob
import os

# Load list of suitable filenames
valid_filenames = set(sorted(suitable_logs_list))

# List and filter .xes files
xes_file_paths = glob.glob('/home/vco/Datasets/Collection_Event_Logs/*.xes')
xes_file_paths = [os.path.abspath(p) for p in xes_file_paths]
print(sorted(xes_file_paths))
xes_file_paths = [p for p in xes_file_paths if os.path.basename(p) in valid_filenames]

# List and filter .csv files, excluding Metadata.csv
csv_file_paths = glob.glob('/home/vco/Datasets/Collection_Event_Logs/*.csv')
csv_file_paths = [p for p in csv_file_paths if not p.endswith('Metadata.csv')]
csv_file_paths = [os.path.abspath(p) for p in csv_file_paths]
csv_file_paths = [p for p in csv_file_paths if os.path.basename(p) in valid_filenames]

# Print results
print("Filtered .xes files:")
print(xes_file_paths)

print("Filtered .csv files:")
print(csv_file_paths)


In [ ]:
suitable_logs_list.remove('BPI_Challenge_2012.csv')
suitable_logs_list.append('BPI_Challenge_2012.xes')

In [ ]:
sorted(suitable_logs_list)

In [ ]:
xes_file_paths

In [ ]:
# log = pm4py.read_xes('/home/vco/Projects/pm4py-dcr4py/tests/input_data/roadtraffic50traces.xes')
log = pm4py.read_xes('/home/vco/Datasets/Road_Traffic_Fine_Management_Process.xes.gz')

In [ ]:
sepsis = "Sepsis_Cases_Event_Log.xes"
log = pm4py.read_xes(f'/home/vco/Datasets/{sepsis}')

In [ ]:
log = pm4py.read_xes('/home/vco/Datasets/Collection_Event_Logs/BPI_Challenge_2012.xes')

In [ ]:
all_events = set(log['concept:name'].unique())

In [ ]:
events_with_detailed_lifecycles = set()
subset_logs_for_event = {}
for event in all_events:
    subset_log = log[log['concept:name']==event][['case:concept:name','lifecycle:transition','time:timestamp']]
    lifecycle_transitions = subset_log['lifecycle:transition'].unique()
    if len(lifecycle_transitions) > 1:
        print(event, lifecycle_transitions)
        events_with_detailed_lifecycles.add(event)
        subset_log.rename({'lifecycle:transition':'concept:name'},axis=1,inplace=True)
        subset_logs_for_event[event] = subset_log

In [ ]:
only_complete_lifecycle_log = log[log['lifecycle:transition']=='COMPLETE']

In [ ]:
only_complete_lifecycle_log

In [ ]:
lifecycle_graphs_for_event = {}
for event, subset_log in subset_logs_for_event.items():
    graph, timings = pm4py.discover_dcr(subset_log, post_process={'timed'})
    lifecycle_graphs_for_event[event] = graph

In [ ]:
for event, graph in lifecycle_graphs_for_event.items():
    print(event)
    pm4py.view_dcr(graph)

In [ ]:
from copy import deepcopy

In [ ]:
domain_knowledge_abstraction_log = deepcopy(log)
domain_knowledge_abstraction_log['concept:name'] = domain_knowledge_abstraction_log['concept:name'].apply(
    lambda x: x.split('_')[0])
subprocess_graph, subprocess_timings = pm4py.discover_dcr(domain_knowledge_abstraction_log, post_process={'timed'})

In [ ]:
pm4py.view_dcr(subprocess_graph)

In [ ]:
lifecycle_in_event_log = deepcopy(log)
lifecycle_in_event_log['concept:name'] = lifecycle_in_event_log['concept:name'] + '+' + lifecycle_in_event_log['lifecycle:transition']

In [ ]:
lifecycle_in_event_graph, _ = pm4py.discover_dcr(lifecycle_in_event_log, post_process={'nesting','timed'},nest_variant=NestVariants.CHOICE)

In [ ]:
graph, timings = pm4py.discover_dcr(log,post_process={'nesting','timed'},nest_variant=NestVariants.CHOICE)

In [ ]:
lifecycle_in_event_graph = handle_timed_condition_subprocess(lifecycle_in_event_graph)

In [ ]:
graph = handle_timed_condition_subprocess(graph)

In [ ]:
def lifecycle_graph_update(graph,lifecycle_graphs_for_event,subprocess_graph):
    events_by_prefix = {
        sp_prefix: set([e for e in graph.events if e.startswith(sp_prefix)])
        for sp_prefix in ['A', 'W', 'O']
    }
    # events_by_prefix['O'].remove('O_SENT')
    graph.events = graph.events.union(set(['A', 'W', 'O']))
    graph.labels = graph.labels.union(set(['A', 'W', 'O']))
    graph.label_map['A'] = 'A'
    graph.label_map['W'] = 'W'
    graph.label_map['O'] = 'O'
    graph.conditions = {**graph.conditions, **subprocess_graph.conditions}
    graph.timedconditions = {**graph.timedconditions, **subprocess_graph.timedconditions}
    graph.marking.included = graph.marking.included.union(set(['A', 'W', 'O']))
    graph.subprocesses = {**graph.subprocesses, **events_by_prefix}
    for lf_subprocess, lf_graph in lifecycle_graphs_for_event.items():
        graph.events.add(lf_subprocess)
        graph.labels.add(lf_subprocess)
        graph.label_map[lf_subprocess] = lf_subprocess
        graph.marking.included.add(lf_subprocess)
        graph.subprocesses[lf_subprocess] = set([f"{lf_subprocess}+{e}" for e in lf_graph.events])
        graph.timedconditions = {**graph.timedconditions, **lf_graph.timedconditions}
        graph.conditions = {**graph.conditions, **lf_graph.conditions}
        graph.responses = {**graph.responses, **lf_graph.responses}
        graph.includes = {**graph.includes, **lf_graph.includes}
        graph.excludes = {**graph.excludes, **lf_graph.excludes}
    return graph

In [ ]:
graph

In [ ]:
lifecycle_in_event_graph = lifecycle_graph_update(lifecycle_in_event_graph,lifecycle_graphs_for_event,subprocess_graph)

In [ ]:
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
dcr_exporter.apply(lifecycle_in_event_graph, '/home/vco/Projects/delay-deadline-miner/models/lifecycle_in_event_graph2.xml')

# LLM labels my subprocesses

In [ ]:
import os
from http.client import responses
import json
import pandas as pd
from openai import OpenAI, AzureOpenAI
import json
from itertools import product

class LLMSubprocessLabeler(object):

    azure_client = AzureOpenAI(
        azure_endpoint="https://gdprscenariosoai.openai.azure.com/",
        api_key="9d28285b90864a218981162f600a2661",
        api_version="2024-08-01-preview"
    )

    def __init__(self):
        json_answer = {
          "label": ""
        }
        self.system_prompt = f"""You return a single word label for the subprocess. ONLY WRITE THE ANSWER!!! ALWAYS RETURN THE ANSWER IN THE JSON OBJECT SPECIFIED:
        {json_answer}"""

    def create_user_prompt(self,
                    subprocess_activities_list = [],
                    preceded_list = [],
                    succeeded_list = [],
                ):
        no_of_activities = len(subprocess_activities_list)
        preceded = f""" They are preceded by {preceded_list}.""" if len(preceded_list) > 0 else ""
        succeeded = f""" They are succeeded by {succeeded_list}.""" if len(succeeded_list) > 0 else ""
        json_answer = {
          "label": ""
        }
        prompt = f"""
        In a process model I have {no_of_activities} activities labeled: {subprocess_activities_list}.{preceded}{succeeded} As much as possible try to provide a single word label.
        Create a label for the subprocess which contains the {no_of_activities} activities. ONLY WRITE THE ANSWER!!! ALWAYS RETURN THE ANSWER IN THIS JSON OBJECT:
        {json_answer}
        """
        return prompt

    def get_llm_answer(self, input_prompt):
        completion = self.azure_client.chat.completions.create(
        model="gpt-35-turbo",
        messages=[
                {"role": "system", "content": self.system_prompt},
                {
                    "role": "user",
                    "content": input_prompt
                }
            ]
        )
        res = completion.choices[0].message.to_json()
        return json.loads(res)['content']

chat = LLMSubprocessLabeler()
input_prompt = chat.create_user_prompt(original_dcr.nestedgroups['Choice1'],original_dcr.conditions['Choice1'])
res = chat.get_llm_answer(input_prompt)
input_prompt_2 = chat.create_user_prompt(original_dcr.nestedgroups['Choice2'])
res_2 = chat.get_llm_answer(input_prompt_2)

# Old Code

In [ ]:
# from copy import deepcopy
#
# import pandas as pd
# from typing import Optional, Any, Union, Dict
# import pm4py
# from pm4py.objects.dcr.obj import DcrGraph
# from pm4py.util import exec_utils, constants, xes_constants
# from pm4py.objects.dcr.timed.obj import TimedDcrGraph
# from pm4py.objects.log.obj import EventLog
#
# class TimeMining:
#     """
#     The TimeMining provides a simple algorithm to mine timing data of an event log for DCR graphs
#
#     After initialization, user can call mine(log, G, parameters), which will return a DCR Graph with time.
#
#     Attributes
#     ----------
#     graph: Dict[str,Any]
#
#     Methods
#     -------
#     mine(log, G, parameters)
#
#     Notes
#     ------
#     *
#     """
#     def __init__(self):
#         self.idx_list = []
#         self.timing_dict = {"conditionsForDelays": {}, "responseToDeadlines": {}}
#
#
#     def get_log_with_pair(self, event_log, e1, e2):
#         '''
#         when selecting the case ids (cids) here there is a difference when taking
#         strictly less than < and strictly less than or equal <=
#         Less than or equal <= allows for instant executions (so a time of 0 between events e1 and e2)
#         '''
#         first_e1 = event_log[event_log['concept:name'] == e1].groupby('case:concept:name')[
#             ['case:concept:name', 'time:timestamp']].first().reset_index(drop=True)
#         subset_is_in = first_e1.merge(event_log, on='case:concept:name', how='inner', suffixes=('_e1', ''))
#         cids = subset_is_in[
#             ((subset_is_in['time:timestamp_e1'] <= subset_is_in['time:timestamp']) & (subset_is_in['concept:name'] == e2))][
#             'case:concept:name'].unique()
#         return event_log[event_log['case:concept:name'].isin(cids)].copy(deep=True)
#
#
#     def get_delta_between_events(self, filtered_df, event_pair, rule=None):
#         e1 = event_pair[0]
#         e2 = event_pair[1]
#         filtered_df = filtered_df[['case:concept:name', 'concept:name', 'time:timestamp']]
#         filtered_df = filtered_df[filtered_df['concept:name'].isin(event_pair)]
#         filtered_df['time:timestamp'] = pd.to_datetime(filtered_df['time:timestamp'], utc=True)
#         deltas = []
#         deltas_map = {}
#         # for idx, g in filtered_df[filtered_df['concept:name'].isin([e1, e2])].groupby('case:concept:name'):
#         for idx, g in filtered_df.groupby('case:concept:name'):
#             g = g.sort_values(by='time:timestamp').reset_index(drop=True)
#             g['time:timestamp:to'] = g['time:timestamp'].shift(-1)
#             g['concept:name:to'] = g['concept:name'].shift(-1)
#             temp_df = deepcopy(g)
#             res = []
#             if rule == 'RESPONSE':
#                 g_e1 = deepcopy(g[g['concept:name'] == e1])
#                 if len(g_e1) >= 1:
#                     g_e1 = g_e1.reset_index(drop=False)
#                     g_e1['index_below'] = g_e1['index'].shift(-1)
#                     g_e1 = g_e1[((g_e1['index_below'] - g_e1['index']) == 1)]
#                     g_e1['delta'] = g_e1['time:timestamp:to'] - g_e1['time:timestamp']
#                     res.extend(g_e1['delta'])
#                 temp_df = temp_df[
#                     (temp_df['concept:name'] == e1) & (temp_df['concept:name:to'] == e2)]
#                 temp_df['delta'] = temp_df['time:timestamp:to'] - temp_df['time:timestamp']
#                 res.extend(temp_df['delta'])
#             elif rule == 'CONDITION':
#                 temp_df = temp_df[
#                     (temp_df['concept:name'] == e1) & (temp_df['concept:name:to'] == e2)]
#                 temp_df['delta'] = temp_df['time:timestamp:to'] - temp_df['time:timestamp']
#                 res.extend(temp_df['delta'])
#             else:
#                 temp_df = temp_df[
#                     (temp_df['concept:name'] == e1) & (temp_df['concept:name:to'] == e2)]
#                 temp_df['delta'] = temp_df['time:timestamp:to'] - temp_df['time:timestamp']
#                 res.extend(temp_df['delta'])
#             deltas.extend(res)
#             for i in set(res):
#                 if i not in deltas_map:
#                     deltas_map[i] = set()
#                 deltas_map[i].add(idx)
#         return deltas, deltas_map
#
#
#     def get_filter_map(self, log: Union[pd.DataFrame, EventLog], graph):
#         # perform mining on event logs
#         if not isinstance(log, pd.DataFrame):
#             log = pm4py.convert_to_dataframe(log)
#         activities = log['case:concept:name'].unique()
#         print(log.index)
#
#         timing_input_dict = {'CONDITION': set(), 'RESPONSE': set()}
#         for e1 in graph.conditions.keys():
#             for e2 in graph.conditions[e1]:
#                 timing_input_dict['CONDITION'].add((e2, e1))
#
#         for e1 in graph.responses.keys():
#             for e2 in graph.responses[e1]:
#                 timing_input_dict['RESPONSE'].add((e1, e2))
#
#         res_filter_map = {}
#         timings = {}
#         for rule, event_pairs in timing_input_dict.items():
#             for event_pair in event_pairs:
#                 if event_pair[0] in activities and event_pair[1] in activities:
#                     data, filter_map = self.get_timings_one_relation(log,event_pair[0], event_pair[1], rule)
#                     timings[(rule, event_pair[0], event_pair[1])] = data
#                     res_filter_map[(rule, event_pair[0], event_pair[1])] = filter_map
#
#         return timings, res_filter_map
#
#     def get_timings_one_relation(self,log, e1, e2, rule):
#         filtered_df = self.get_log_with_pair(log, e1, e2)
#         data, filter_map = self.get_delta_between_events(filtered_df, (e1,e2), rule)
#         return data, filter_map

In [ ]:
# tm = TimeMining()
# tm.get_filter_map(log,graph)